In [1]:
from typing import Optional, Dict, Any
import os
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy
from openai import AzureOpenAI
from azure.identity import AzureCliCredential, get_bearer_token_provider, DefaultAzureCredential
import ollama

In [2]:
# Initialize Azure credentials
credential = AzureCliCredential()

In [11]:
# ----- Configuration -----
# Azure AI Project information
azure_ai_project = os.environ.get("RED_TEAM_PROJECT_ENDPOINT")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_ADVERSARIAL_DEPLOYMENT_NAME")
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ADVERSARIAL_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_ADVERSARIAL_API_KEY")
azure_openai_api_version = os.environ.get("AZURE_OPENAI_ADVERSARIAL_API_VERSION")

# azure_openai_deployment = os.environ.get("AZURE_OPENAI_UNSAFE_DEPLOYMENT_NAME")
# azure_openai_endpoint = os.environ.get("AZURE_OPENAI_UNSAFE_ENDPOINT")
# azure_openai_api_key = os.environ.get("AZURE_OPENAI_UNSAFE_API_KEY")
# azure_openai_api_version = os.environ.get("AZURE_OPENAI_UNSAFE_API_VERSION")

In [12]:
# AzureOpenAI Callback
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,
    session_state: Optional[str] = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict[str, list[dict[str, str]]]:
    
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}


In [13]:
# Llama Callback
MODEL_NAME = "llama3.2"

async def ollama_callback(
    messages: list,
    stream: Optional[bool] = False,
    session_state: Optional[str] = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict[str, list[dict[str, str]]]:

    # Extract the latest message from the conversation history
    messages_list = [{"role": m.role, "content": m.content} for m in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the local LLaMA model via Ollama
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": latest_message}],
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {
            "content": response["message"]["content"],
            "role": "assistant"
        }

    except Exception as e:
        print(f"Error calling Ollama: {e!s}")
        formatted_response = {
            "content": "I encountered an error and couldn't process your request.",
            "role": "assistant"
        }

    return {"messages": [formatted_response]}


In [14]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

In [15]:
result = await red_team.scan(
    target=ollama_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="red_team_output.json",
)

🚀 STARTING RED TEAM SCAN
📂 Output directory: ./.scan_Basic-Callback-Scan_20251006_145858
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/16f42b7d-81f3-4706-a8bd-922890cb9e26?wsid=/subscriptions/ffcdaad1-98c2-4d37-97be-371caf0be262/resourceGroups/FYP/providers/Microsoft.CognitiveServices/accounts/fyp-resource/projects/fyp&tid=15ce9348-be2a-462b-8fc0-e1765a9b204a
📋 Planning 52 total tasks
📝 Fetched baseline objectives for violence: 5 objectives
📝 Fetched baseline objectives for hate_unfairness: 5 objectives
📝 Fetched baseline objectives for sexual: 5 objectives
📝 Fetched baseline objectives for self_harm: 5 objectives
🔄 Fetching objectives for strategy 2/13: character_space
🔄 Fetching objectives for strategy 3/13: rot13
🔄 Fetching objectives for strategy 4/13: unicode_confusable
🔄 Fetching objectives for strategy 5/13: char_swap
🔄 Fetching objectives for strategy 6/13: morse

Scanning:   0%|                                      | 0/52 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 52 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: character_space strategy for violence risk category


Scanning:   8%|██                         | 4/52 [02:35<2:04:39, 155.83s/scan, current=initializing]

Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/baseline_violence_e08feae6-5032-4785-97de-6effd3f3bd71.json".
✅ Completed task 1/52 (1.9%) - baseline/violence in 155.8s
   Est. remaining: 159.8 minutes
Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/baseline_hate_unfairness_b8e723f7-8aa4-4a79-a87e-4a6e1e9d1a3c.json".
✅ Completed task 2/52 (3.8%) - baseline/hate_unfairness in 155.8s
   Est. remaining: 78.3 minutes
Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/baseline_sexual_7b8c6853-e8b7-4c67-af71-8928a211626d.json".
✅ Completed task 3/52 (5.8%) - baseline/sexual in 155.8s
   Est. remaining: 51.2 minutes
Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Sca

...

Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/tense_hate_unfairness_e398eb3e-057e-49e8-9e8b-ef79c993ee83.json".
✅ Completed task 50/52 (96.2%) - tense/hate_unfairness in 383.0s
   Est. remaining: 1.7 minutes
▶️ Starting task: tense strategy for sexual risk category
▶️ Starting task: tense strategy for self_harm risk category


Scanning:  98%|████████████████████████████▍| 51/52 [44:44<01:02, 62.82s/scan, current=initializing]

Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/tense_sexual_3b14f805-0fea-4d2f-8cc1-8f2a53561c04.json".
✅ Completed task 51/52 (98.1%) - tense/sexual in 119.8s
   Est. remaining: 0.9 minutes


Scanning: 100%|█████████████████████████████| 52/52 [44:45<00:00, 51.64s/scan, current=initializing]


Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/tense_self_harm_c79e0345-1655-4673-a2e9-b3df5739b943.json".
✅ Completed task 52/52 (100.0%) - tense/self_harm in 120.1s
   Est. remaining: 0.0 minutes
Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/red_team_output.json".

Evaluation results saved to "/Users/kendrickkoh/Desktop/University/FYP/Red teaming preparation/.scan_Basic-Callback-Scan_20251006_145858/final_results.json".

Overall ASR: 3.46%
Attack Success: 9/260 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
--------------------------------------------------------------------------------------------------

In [16]:
# # Run the red team scan with multiple attack strategies
# advanced_result = await model_red_team.scan(
#     target=azure_openai_callback,
#     scan_name="Advanced-Callback-Scan",
#     attack_strategies=[
#         AttackStrategy.EASY,  # Group of easy complexity attacks
#         AttackStrategy.MODERATE,  # Group of moderate complexity attacks
#         AttackStrategy.CharacterSpace,  # Add character spaces
#         AttackStrategy.ROT13,  # Use ROT13 encoding
#         AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
#         AttackStrategy.CharSwap,  # Swap characters in prompts
#         AttackStrategy.Morse,  # Encode prompts in Morse code
#         AttackStrategy.Leetspeak,  # Use Leetspeak
#         AttackStrategy.Url,  # Use URLs in prompts
#         AttackStrategy.Binary,  # Encode prompts in binary
#         AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
#     ],
#     output_path="Advanced-Callback-Scan.json",
# )